#### Load packages

In [1]:
import pandas as pd

#### Load data

In [2]:
data = pd.read_csv('./../data/raw/big-mac-source-data-v2.csv')
data['date'] = pd.to_datetime(data['date'])

In [3]:
data.head()

,name,iso_a3,currency_code,local_price,dollar_ex,GDP_dollar,GDP_local,date
0,Argentina,ARG,ARS,2.50,1.00,8709.072,8709.072,2000-04-01
1,Australia,AUS,AUD,2.59,1.68,21746.809,33698.764,2000-04-01
2,Brazil,BRA,BRL,2.95,1.79,3501.438,6351.375,2000-04-01
3,Canada,CAN,CAD,2.85,1.47,22340.553,33191.468,2000-04-01
4,Switzerland,CHE,CHF,5.90,1.70,41859.215,62879.029,2000-04-01


#### Get latest data

In [4]:
big_mac_countries = [
    'ARG', 'AUS', 'BRA', 'GBR', 'CAN', 'CHL', 'CHN', 'CZE', 'DNK',
    'EGY', 'HKG', 'HUN', 'IDN', 'ISR', 'JPN', 'MYS', 'MEX', 'NZL',
    'NOR', 'PER', 'PHL', 'POL', 'RUS', 'SAU', 'SGP', 'ZAF', 'KOR',
    'SWE', 'CHE', 'TWN', 'THA', 'TUR', 'ARE', 'USA', 'COL', 'CRI',
    'PAK', 'LKA', 'UKR', 'URY', 'IND', 'VNM', 'GTM', 'HND', 'VEN',
    'NIC', 'AZE', 'BHR', 'HRV', 'JOR', 'KWT', 'LBN', 'MDA', 'OMN',
    'QAT', 'ROU', 'EUZ'
]

In [5]:
latest_date = data['date'].max()
latest_data = data[(data['date'] == latest_date)
                & (data['iso_a3'].isin(big_mac_countries))].reset_index(drop=True)

#### Add Kazakhstan data

In [6]:
latest_data = latest_data.append(
    {
        'name' : 'Kazakhstan',
        'iso_a3' : 'KAZ',
        'currency_code' : 'KZT',
        'local_price' : 1050.0,
        'dollar_ex' : 479.86,
        'GDP_dollar' : None,
        'GDP_local' : None,
        'date' : latest_date,
    },
    ignore_index=True,
)

/tmp/ipykernel_10436/1932806503.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latest_data = latest_data.append(


#### Calculate dollar price

In [7]:
latest_data['dollar_price'] = latest_data['local_price']/latest_data['dollar_ex']

#### Calculate the Big Mac Index

Select our base currencies: the US dollar (USD), Euro (EUR), British pound (GBP), Japanese yen (JPY), Chinese yuan (CNY) and Kazakhstani tenge (KZT).

In [8]:
base_currencies = ['USD', 'EUR', 'GBP', 'JPY', 'CNY', 'KZT']

In [9]:
for currency in base_currencies:
    latest_data[currency] = latest_data['dollar_price'] / latest_data[latest_data['currency_code'] == currency]['dollar_price'].values-1

In [10]:
latest_data.head()

,name,iso_a3,currency_code,local_price,dollar_ex,GDP_dollar,GDP_local,date,dollar_price,USD,EUR,GBP,JPY,CNY,KZT
0,United Arab Emirates,ARE,AED,18.0,3.673050,42883.686,157490.338,2022-07-01,4.900559,-0.048435,0.028432,0.103728,0.732348,0.377741,1.239602
1,Argentina,ARG,ARS,590.0,129.115000,10658.458,1013547.474,2022-07-01,4.569570,-0.112705,-0.041029,0.029181,0.615343,0.284687,1.088337
2,Australia,AUS,AUD,6.7,1.448436,63529.289,84505.238,2022-07-01,4.625680,-0.101810,-0.029254,0.041818,0.635178,0.300462,1.113980
3,Azerbaijan,AZE,AZN,4.7,1.698250,5397.928,9176.478,2022-07-01,2.767555,-0.462611,-0.419200,-0.376678,-0.021669,-0.221931,0.264799
4,Bahrain,BHR,BHD,1.6,0.377000,26135.777,9827.052,2022-07-01,4.244032,-0.175916,-0.109347,-0.044138,0.500265,0.193165,0.939563


In [33]:
import plotly.express as px
import plotly.graph_objects as go

data = latest_data.sort_values(by = ['USD'],
                    ascending = True).reset_index(drop=True)
fig = go.Figure()

# Draw points
fig.add_trace(go.Scatter(y = data["USD"], 
                          x = data["name"],
                          mode = 'markers',
                          marker_color ='darkblue',
                          marker_size  = 10))

# Draw lines
for i in range(0, len(data)):
               fig.add_shape(type='line',
                              y0 = 0, x0 = i,
                              y1 = data["USD"][i],
                              x1 = i,
                              line=dict(color='crimson' if data["USD"][i] < 0 else 'blue', width = 3))

# Set x-axes range
fig.update_xaxes(title = 'The Big Mac index for USD')

# Set y-axes as percentage
fig.update_yaxes(tickformat=".1%")

# Set figure size
fig.update_layout(
    autosize=False,
    width=1200,
    height=800,
    paper_bgcolor="rgba(255,255,255,1)", 
    plot_bgcolor="rgba(255,255,255,1)"
)

fig.show()

In [34]:
latest_data.to_csv('./../data/cleaned/big-mac-source-data-plus-kaz.csv', index=False)